# Cube CSV Database Searcher

## The entire database exists. We just need to sift through it. Somehow.

So yeah this notebook is simple. There is a CubeCSVDatabase I have with .csv files for *all* the manageable cubes. 

We already know how to find a cube if we already *have* the cube number and flyby number, so the thing we need to do is create a file that *holds* all that information. Just a LIST of cubes and flybys. 

In [15]:
#DO NOT RUN THIS, IT'LL JUST EXPAND THE FILE! 
#DELETE THE FILE BEFORE RECREATING! BUT BE SURE YOU WANT TO!

#Imports:
from os import listdir
from os.path import isfile, join

#The filepath for the database:
datapath = r"C:\Users\deran\Desktop\CubeCSVDatabase\T" #followed by the number or letter.
path = r"C:\Users\deran\Desktop\CubeCSVDatabase\T114\CM_1826052615_1.cub.csv" #as an example of a full thing.

with open("cubeList.csv", 'a') as dataEntry: #a for append.
    flypath = datapath+"a"
    allFiles = [f for f in listdir(flypath) if isfile(join(flypath, f))]
    for item in allFiles:
        if (item[-7::1] == "cub.csv"):
            if (item[-11::1] != "geo.cub.csv"):
                dataEntry.write(str(item[:15:])+","+str(flypath[-2::])+","+ "\n")
    flypath = datapath+"b"
    allFiles = [f for f in listdir(flypath) if isfile(join(flypath, f))]
    for item in allFiles:
        if (item[-7::1] == "cub.csv"):
            if (item[-11::1] != "geo.cub.csv"):
                dataEntry.write(str(item[:15:])+","+str(flypath[-2::])+","+ "\n")
    flypath = datapath+"c"
    allFiles = [f for f in listdir(flypath) if isfile(join(flypath, f))]
    for item in allFiles:
        if (item[-7::1] == "cub.csv"):
            if (item[-11::1] != "geo.cub.csv"):
                dataEntry.write(str(item[:15:])+","+str(flypath[-2::])+","+ "\n")
    flypath = datapath+"0"
    allFiles = [f for f in listdir(flypath) if isfile(join(flypath, f))]
    for item in allFiles:
        if (item[-7::1] == "cub.csv"):
            if (item[-11::1] != "geo.cub.csv"):
                dataEntry.write(str(item[:15:])+","+str(flypath[-2::])+","+ "\n")
#Now that we're past the weird flybys, we can use numbers.
    num = 3
    while (num <= 126):
        flypath = datapath+str(num)
        allFiles = [f for f in listdir(flypath) if isfile(join(flypath, f))]
        for item in allFiles:
            if (item[-7::1] == "cub.csv"):
                if (item[-11::1] != "geo.cub.csv"):
                    if (num < 10):
                        dataEntry.write(str(item[:15:])+","+str(flypath[-2::])+","+ "\n")
                    elif(num < 100):
                        dataEntry.write(str(item[:15:])+","+str(flypath[-3::])+","+ "\n")
                    else:
                        dataEntry.write(str(item[:15:])+","+str(flypath[-4::])+","+ "\n")
        num=num+1




So, since we have a list of cubes and flybys, we can now go through them and create a new list, one that contains the flybys and cube numbers of those that are easily removed. The easy removal can be done by dimensions: if it has less than 5 pixels in one dimension, we nuke it, as we don't want noodles or noodle-like images. Based on the fact that we originally used 3x3 pixel areas to input data.



In [33]:
#Imports
import csv
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy.misc
from scipy import ndimage

#First, read in the data into a matrix. 
cubeList = []
with open('cubeList.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        paired = ["A","A"]
        paired[0] = row[0]
        paired[1] = row[1]
        cubeList.append(paired)
        line_count += 1

newCubeList = []
#Now, for every item in cubeList, we find the file it poitns to and read it in. 
#Then we examine those files for *something*. Whatever it is. 
#This time we're nuking noodles, so the test is simple and won't require iterations.
for item in cubeList:
    err = 0 #No error by default.
    
    #This code is copy adapted from the VIMS Cube Visualisation Interface Notebook. 
    #It is complicated.
    filepath = "C:\\Users\\deran\\Desktop\\CubeCSVDatabase\\" + item[1] + "\\" + item[0] + ".cub.csv"

    #Now we extract the axes file as well...
    cubeAxesfp = filepath.removesuffix(".csv") + ".axes.csv"
    
    #And you know what why not all the others as well.
    cubeGeofpIR = filepath.removesuffix(".cub.csv") + "_ir_geo.cub.csv"
    cubeGeofpIRaxes = filepath.removesuffix(".cub.csv") + "_ir_geo.cub.axes.csv"
    #Basically, we don't use axesfp if it's not needed.
    
    #Skeleton code nabbed from https://realpython.com/python-csv/
    #We need to be able to handle both local and other cubes. 
    #Pyvims can help us, but as I don't want to redo all the code we'll find a way to store
    #it in the way the code understands.
    
    #Step 1: use the axes to determine the size of what we're dealing with.
    xAxisCube = []
    yAxisCube = []
    zAxisCube = []
    
    xAxisGeoIR = []
    yAxisGeoIR = []
    zAxisGeoIR = []
    
    xAxisGeoVIS = []
    yAxisGeoVIS = []
    zAxisGeoVIS = []

    try:
        with open(cubeAxesfp) as csv_file: #remember to tab.
                csv_reader = csv.reader(csv_file, delimiter=',')
                line_count = 0
                for row in csv_reader:
                    i = 0
                    L = len(row)
                    while (i < L-1):
                        if (line_count == 0):
                            xAxisCube.append(row[i])
                        elif (line_count == 1):
                            yAxisCube.append(row[i])
                        elif (line_count == 2):
                            zAxisCube.append(row[i])
                        i = i+1
                    line_count += 1
    except:
        err = 1 #whoops.        
    try:
        with open(cubeGeofpIRaxes) as csv_file: #remember to tab.
                csv_reader = csv.reader(csv_file, delimiter=',')
                #No need to waste time doing things if we don't need them.
                line_count = 0
    except:
        err = 1 #whoops.
        
    #We now have an x, y, and z axis. x and y axes are just ordinal, but the z axis contains wavelength in microns.
    #The lengths of these arrays tell us how to extract the data.
    
    cubeData = [[[0 for x in range(len(zAxisCube))] for x in range(len(yAxisCube))] for x in range(len(xAxisCube))]
    geoIRData = [[[0 for x in range(len(zAxisGeoIR))] for x in range(len(yAxisGeoIR))] for x in range(len(xAxisGeoIR))]
    
    #The above holds the data of the cube itself. 
    try:
        with open(filepath) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            #No need to waste time doing things if we don't need them.
    except:
        err = 1 #whoops.
    try:
        with open(cubeGeofpIR) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            #No need to waste time doing things if we don't need them.
    except:
        err = 1 #Whoops.
    #The data is now read in.
    #Now we can do stuff with it. So for the test let's just print a value from it.
    if (err == 0):
        if ((len(xAxisCube) > 4) and (len(yAxisCube) > 4)):
            newCubeList.append([item[0],item[1]])
    else:
        print("Error on", item[0], item[1])
        
print(newCubeList)

Error on CM_1477314984_3 Ta
Error on CM_1477324010_3 Ta
Error on CM_1477519241_4 Ta
Error on CM_1481360933_4 Tb
Error on CM_1481364553_4 Tb
Error on CM_1481620217_4 Tb
Error on CM_1467426144_5 T0
Error on CM_1492356134_3 T5
Error on CM_1503380568_3 T6
Error on CM_1514298007_3 T9
Error on CM_1516040056_3 T10
Error on CM_1519692349_3 T11
Error on CM_1521302464_3 T12
Error on CM_1519751704_1 T13
Error on CM_1526768731_3 T14
Error on CM_1526771512_3 T14
Error on CM_1526826445_1 T14
Error on CM_1530528672_1 T15
Error on CM_1532227988_2 T16
Error on CM_1536352432_1 T17
Error on CM_1536352436_1 T17
Error on CM_1536352440_1 T17
Error on CM_1536352445_1 T17
Error on CM_1536352449_1 T17
Error on CM_1536352453_1 T17
Error on CM_1536352457_1 T17
Error on CM_1536352461_1 T17
Error on CM_1536352465_1 T17
Error on CM_1536352469_1 T17
Error on CM_1536352473_1 T17
Error on CM_1536352477_1 T17
Error on CM_1536352481_1 T17
Error on CM_1536352485_1 T17
Error on CM_1536352489_1 T17
Error on CM_1536352493_1

In [35]:
print(len(newCubeList))
print(len(cubeList))

18569
63521


In [39]:
#Imports
import csv
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy.misc
from scipy import ndimage

newCubeList2 = []
#Now, for every item in cubeList, we find the file it poitns to and read it in. 
#Then we examine those files for *something*. Whatever it is. 
#This time we're nuking noodles, so the test is simple and won't require iterations.
for item in newCubeList:
    err = 0 #No error by default.
    
    #This code is copy adapted from the VIMS Cube Visualisation Interface Notebook. 
    #It is complicated.
    filepath = "C:\\Users\\deran\\Desktop\\CubeCSVDatabase\\" + item[1] + "\\" + item[0] + ".cub.csv"

    #Now we extract the axes file as well...
    cubeAxesfp = filepath.removesuffix(".csv") + ".axes.csv"
    
    #And you know what why not all the others as well.
    cubeGeofpIR = filepath.removesuffix(".cub.csv") + "_ir_geo.cub.csv"
    cubeGeofpIRaxes = filepath.removesuffix(".cub.csv") + "_ir_geo.cub.axes.csv"
    #Basically, we don't use axesfp if it's not needed.
    
    #Skeleton code nabbed from https://realpython.com/python-csv/
    #We need to be able to handle both local and other cubes. 
    #Pyvims can help us, but as I don't want to redo all the code we'll find a way to store
    #it in the way the code understands.
    
    #Step 1: use the axes to determine the size of what we're dealing with.
    xAxisCube = []
    yAxisCube = []
    zAxisCube = []
    
    xAxisGeoIR = []
    yAxisGeoIR = []
    zAxisGeoIR = []
    
    xAxisGeoVIS = []
    yAxisGeoVIS = []
    zAxisGeoVIS = []


    with open(cubeAxesfp) as csv_file: #remember to tab.
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            for row in csv_reader:
                i = 0
                L = len(row)
                while (i < L-1):
                    if (line_count == 0):
                        xAxisCube.append(row[i])
                    elif (line_count == 1):
                        yAxisCube.append(row[i])
                    elif (line_count == 2):
                        zAxisCube.append(row[i])
                    i = i+1
                line_count += 1
  

    with open(cubeGeofpIRaxes) as csv_file: #remember to tab.
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            for row in csv_reader:
                i = 0
                L = len(row)
                while (i < L-1):
                    if (line_count == 0):
                        xAxisGeoIR.append(row[i])
                    elif (line_count == 1):
                        yAxisGeoIR.append(row[i])
                    elif (line_count == 2):
                        zAxisGeoIR.append(row[i])
                    i = i+1
                line_count += 1

        
    #We now have an x, y, and z axis. x and y axes are just ordinal, but the z axis contains wavelength in microns.
    #The lengths of these arrays tell us how to extract the data.
    
    cubeData = [[[0 for x in range(len(zAxisCube))] for x in range(len(yAxisCube))] for x in range(len(xAxisCube))]
    geoIRData = [[[0 for x in range(len(zAxisGeoIR))] for x in range(len(yAxisGeoIR))] for x in range(len(xAxisGeoIR))]
    
    #The above holds the data of the cube itself. 

    with open(filepath) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        #No need to actually read in data, as it doesn't help us pair things down.

    with open(cubeGeofpIR) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        i, j, k = 0, 0, 0
        for row in csv_reader:
            while (i < len(xAxisGeoIR)):
                geoIRData[i][j][k] = float(row[i])
                if (math.isnan(geoIRData[i][j][k])):
                    geoIRData[i][j][k] = 0 #We set nans to zero to allow plotting to take place, careful!
                elif (geoIRData[i][j][k] < -1000):
                    geoIRData[i][j][k] = 0 #The default value is an extremely negative number. Scrub it.
                i = i + 1
            i = 0
            j = j + 1
            if (j >= len(yAxisGeoIR)):
                j = 0
                k = k + 1
            line_count += 1

    #The data is now read in.
    #Now we can do stuff with it. So for the test let's just print a value from it.
    #TODAY: Find the Largest Latitude. If it's above 55 degrees, keep the cube. Otherwise... NUKE!
    i,j = 0,0
    bufferVal = 0.0
    while (i < len(xAxisCube)):
        j=0
        while (j < len(yAxisCube)):
            if (bufferVal < geoIRData[i][j][0]):
                bufferVal = geoIRData[i][j][0]
            j=j+1
        i=i+1
    if (bufferVal >= 55.0):
        newCubeList2.append([item[0],item[1]])
        print(bufferVal,item[1])
print(newCubeList2)

59.415344 Ta
59.174149 Ta
66.501724 Ta
65.540085 Ta
63.609135 Ta
69.793465 Ta
66.207901 Ta
65.09285 Ta
65.920807 Ta
64.727943 Ta
64.64994 Ta
61.801777 Ta
67.727531 Ta
60.935661 Ta
59.977501 Ta
58.267239 Ta
70.280807 Ta
68.24839 Ta
67.25248 Ta
67.311142 Ta
72.600334 Ta
64.691582 Ta
67.08889 Ta
66.955467 Ta
68.706924 Ta
73.27095 Ta
64.44133 Ta
65.468575 Ta
72.348198 Ta
67.671364 Ta
66.132706 Ta
70.403709 Ta
72.605103 Ta
67.258675 Ta
67.894028 Ta
68.350868 Ta
56.538479 Ta
56.386387 Ta
73.500793 Ta
66.369507 Ta
89.048195 Ta
88.411255 Ta
88.235535 Ta
88.546303 Ta
63.850742 Tb
62.526611 Tb
64.888794 Tb
57.860455 Tb
60.960785 Tb
57.642574 Tb
59.054745 Tb
56.051743 Tb
70.714241 Tb
66.423752 Tb
65.976448 Tb
65.109695 Tb
64.539635 Tb
63.444622 Tb
64.617676 Tb
64.980438 Tb
58.957878 Tb
60.414783 Tb
59.684673 Tb
56.296204 Tb
62.466797 Tb
60.398727 Tb
60.191879 Tb
69.691666 Tb
68.427483 Tb
64.709564 Tb
66.064636 Tb
62.988506 Tb
62.903973 Tb
66.081932 Tb
75.982246 Tb
74.773895 Tb
68.805298 Tb
65.225

In [43]:
print(len(newCubeList2))


7609


In [51]:
#Imports
import csv
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy.misc
from scipy import ndimage

cubeListLoRes = []
cubeListMedRes = []
cubeListHiRes = []
cubeListExRes = []

#Now, for every item in cubeList, we find the file it poitns to and read it in. 
#Then we examine those files for *something*. Whatever it is. 
#This time we're nuking noodles, so the test is simple and won't require iterations.
for item in newCubeList2:
    err = 0 #No error by default.
    
    #This code is copy adapted from the VIMS Cube Visualisation Interface Notebook. 
    #It is complicated.
    filepath = "C:\\Users\\deran\\Desktop\\CubeCSVDatabase\\" + item[1] + "\\" + item[0] + ".cub.csv"

    #Now we extract the axes file as well...
    cubeAxesfp = filepath.removesuffix(".csv") + ".axes.csv"
    
    #And you know what why not all the others as well.
    cubeGeofpIR = filepath.removesuffix(".cub.csv") + "_ir_geo.cub.csv"
    cubeGeofpIRaxes = filepath.removesuffix(".cub.csv") + "_ir_geo.cub.axes.csv"
    #Basically, we don't use axesfp if it's not needed.
    
    #Skeleton code nabbed from https://realpython.com/python-csv/
    #We need to be able to handle both local and other cubes. 
    #Pyvims can help us, but as I don't want to redo all the code we'll find a way to store
    #it in the way the code understands.
    
    #Step 1: use the axes to determine the size of what we're dealing with.
    xAxisCube = []
    yAxisCube = []
    zAxisCube = []
    
    xAxisGeoIR = []
    yAxisGeoIR = []
    zAxisGeoIR = []
    
    xAxisGeoVIS = []
    yAxisGeoVIS = []
    zAxisGeoVIS = []


    with open(cubeAxesfp) as csv_file: #remember to tab.
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            for row in csv_reader:
                i = 0
                L = len(row)
                while (i < L-1):
                    if (line_count == 0):
                        xAxisCube.append(row[i])
                    elif (line_count == 1):
                        yAxisCube.append(row[i])
                    elif (line_count == 2):
                        zAxisCube.append(row[i])
                    i = i+1
                line_count += 1
  

    with open(cubeGeofpIRaxes) as csv_file: #remember to tab.
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            for row in csv_reader:
                i = 0
                L = len(row)
                while (i < L-1):
                    if (line_count == 0):
                        xAxisGeoIR.append(row[i])
                    elif (line_count == 1):
                        yAxisGeoIR.append(row[i])
                    elif (line_count == 2):
                        zAxisGeoIR.append(row[i])
                    i = i+1
                line_count += 1

        
    #We now have an x, y, and z axis. x and y axes are just ordinal, but the z axis contains wavelength in microns.
    #The lengths of these arrays tell us how to extract the data.
    
    cubeData = [[[0 for x in range(len(zAxisCube))] for x in range(len(yAxisCube))] for x in range(len(xAxisCube))]
    geoIRData = [[[0 for x in range(len(zAxisGeoIR))] for x in range(len(yAxisGeoIR))] for x in range(len(xAxisGeoIR))]
    
    #The above holds the data of the cube itself. 

    with open(filepath) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        #No need to actually read in data, as it doesn't help us pair things down.

    with open(cubeGeofpIR) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        i, j, k = 0, 0, 0
        for row in csv_reader:
            while (i < len(xAxisGeoIR)):
                geoIRData[i][j][k] = float(row[i])
                if (math.isnan(geoIRData[i][j][k])):
                    geoIRData[i][j][k] = 0 #We set nans to zero to allow plotting to take place, careful!
                elif (geoIRData[i][j][k] < -1000):
                    geoIRData[i][j][k] = 0 #The default value is an extremely negative number. Scrub it.
                i = i + 1
            i = 0
            j = j + 1
            if (j >= len(yAxisGeoIR)):
                j = 0
                k = k + 1
            line_count += 1

    #The data is now read in.
    #Now we can do stuff with it. So for the test let's just print a value from it.
    #TODAY: Find the Largest Latitude. If it's above 55 degrees, keep the cube. Otherwise... NUKE!
    i,j = 0,0
    bufferVal = 100.0
    while (i < len(xAxisCube)):
        j=0
        while (j < len(yAxisCube)):
            if (bufferVal > geoIRData[i][j][2] and geoIRData[i][j][2] != 0.0):
                bufferVal = geoIRData[i][j][2]
            j=j+1
        i=i+1
    if (bufferVal <= 50.0 and bufferVal > 0.0): #zero values were NaNs at some point.
        cubeListLoRes.append([item[0],item[1]])
        print(bufferVal,item[1])
        if (bufferVal <= 25.0):
            cubeListMedRes.append([item[0],item[1]])
            if (bufferVal <= 10.0):
                cubeListHiRes.append([item[0],item[1]])
                if (bufferVal <= 5.0):
                    cubeListExRes.append([item[0],item[1]])
print(len(cubeListLoRes), len(cubeListMedRes), len(cubeListHiRes), len(cubeListExRes))

40.136803 Ta
37.928207 Ta
38.189354 Ta
31.918982 Ta
32.103813 Ta
42.753426 Tb
27.067598 Tb
27.18919 Tb
28.797697 Tb
36.284233 T3
42.187305 T4
34.587833 T4
34.538189 T4
32.154728 T4
31.401009 T4
46.062363 T4
46.714314 T4
47.746544 T4
48.376102 T4
49.348198 T4
45.826778 T5
44.546249 T5
41.109745 T5
40.089375 T5
47.828487 T5
37.852974 T5
34.003662 T5
43.353058 T6
42.572361 T6
41.834148 T6
45.744999 T8
45.318188 T8
48.070076 T9
48.464211 T10
39.539612 T10
36.395 T10
35.469162 T10
30.936392 T10
27.701962 T10
42.187706 T10
14.247457 T10
14.925154 T10
26.353138 T10
28.043249 T10
9.986048 T13
47.352039 T14
22.732698 T14
21.790018 T14
19.938238 T14
17.800964 T14
15.373001 T14
15.040093 T14
15.723939 T14
17.081478 T14
17.777811 T14
18.412376 T14
19.121613 T14
47.591434 T15
46.779259 T15
47.956509 T15
7.182207 T15
16.639711 T15
19.573004 T15
11.256183 T15
11.29054 T15
11.298253 T15
48.557686 T15
44.561382 T17
47.978268 T17
48.995232 T17
45.565296 T18
43.160263 T18
42.825748 T18
43.424332 T18
43.7

In [56]:
#Okay now it's time to do some manual inspection. 
#This is because we now need to examine the remaining cubes to see if any are suffering from glitches.
#After this we will return to automation with one of the MASKS. Hopefully.
with open("selectedCubes.csv", 'w') as dataEntry: #a for append.
    i,j,k = 0,0,0 #Iterators for the other lists.
    for item in cubeListLoRes:
        if (i < len(cubeListMedRes)):
            if (item == cubeListMedRes[i]):
                i=i+1
                if (j < len(cubeListHiRes)):
                    if (item == cubeListHiRes[j]):
                        j=j+1
                        if (k < len(cubeListExRes)):
                            if (item == cubeListExRes[k]):
                                k=k+1
                                dataEntry.write(str(item[0]) + "," + str(item[1]) + "," + "Ex" + "," + "\n")
                            else:
                                dataEntry.write(str(item[0]) + "," + str(item[1]) + "," + "Hi" + "," + "\n")
                    else:
                        dataEntry.write(str(item[0]) + "," + str(item[1]) + "," + "Med" + "," + "\n")
            else:
                dataEntry.write(str(item[0]) + "," + str(item[1]) + "," + "Lo" + "," + "\n")
                    


In [57]:
print(cubeListExRes)

[['CM_1548747455_1', 'T24'], ['CM_1548747630_1', 'T24'], ['CM_1548747805_1', 'T24'], ['CM_1567236078_1', 'T35'], ['CM_1567236713_1', 'T35'], ['CM_1654398195_1', 'T69'], ['CM_1654398683_1', 'T69'], ['CM_1654398964_1', 'T69'], ['CM_1654398982_1', 'T69'], ['CM_1654398999_1', 'T69'], ['CM_1654399016_1', 'T69'], ['CM_1654399033_1', 'T69'], ['CM_1654399050_1', 'T69'], ['CM_1654399067_1', 'T69'], ['CM_1654399085_1', 'T69'], ['CM_1654399102_1', 'T69'], ['CM_1654399119_1', 'T69'], ['CM_1654399136_1', 'T69'], ['CM_1654399153_1', 'T69'], ['CM_1654399170_1', 'T69'], ['CM_1654399188_5', 'T69'], ['CM_1654399211_1', 'T69'], ['CM_1654399736_1', 'T69'], ['CM_1654400219_1', 'T69'], ['CM_1721854264_1', 'T85'], ['CM_1721854459_1', 'T85'], ['CM_1721854654_1', 'T85'], ['CM_1721854681_1', 'T85'], ['CM_1721854707_1', 'T85'], ['CM_1721854917_1', 'T85'], ['CM_1721854943_1', 'T85'], ['CM_1721854969_1', 'T85'], ['CM_1753530947_1', 'T93'], ['CM_1753530947_1', 'T93'], ['CM_1753532797_1', 'T93'], ['CM_1757663996_1',

In [ ]:
#Imports
import csv
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy.misc
from scipy import ndimage

#Need a new list to hold all the stuff. 
#After this, each step we can alternate between the two cube list monikers
newCubeList2 = []
for item in newCubeList:    
    newCubeList2.append(item)

newCubeList = []
#Now, for every item in cubeList, we find the file it poitns to and read it in. 
#Then we examine those files for *something*. Whatever it is. 
#This time we're nuking noodles, so the test is simple and won't require iterations.
for item in cubeList:
    err = 0 #No error by default.
    
    #This code is copy adapted from the VIMS Cube Visualisation Interface Notebook. 
    #It is complicated.
    filepath = "C:\\Users\\deran\\Desktop\\CubeCSVDatabase\\" + item[1] + "\\" + item[0] + ".cub.csv"

    #Now we extract the axes file as well...
    cubeAxesfp = filepath.removesuffix(".csv") + ".axes.csv"
    
    #And you know what why not all the others as well.
    cubeGeofpIR = filepath.removesuffix(".cub.csv") + "_ir_geo.cub.csv"
    cubeGeofpIRaxes = filepath.removesuffix(".cub.csv") + "_ir_geo.cub.axes.csv"
    #Basically, we don't use axesfp if it's not needed.
    
    #Skeleton code nabbed from https://realpython.com/python-csv/
    #We need to be able to handle both local and other cubes. 
    #Pyvims can help us, but as I don't want to redo all the code we'll find a way to store
    #it in the way the code understands.
    
    #Step 1: use the axes to determine the size of what we're dealing with.
    xAxisCube = []
    yAxisCube = []
    zAxisCube = []
    
    xAxisGeoIR = []
    yAxisGeoIR = []
    zAxisGeoIR = []
    
    xAxisGeoVIS = []
    yAxisGeoVIS = []
    zAxisGeoVIS = []

    try:
        with open(cubeAxesfp) as csv_file: #remember to tab.
                csv_reader = csv.reader(csv_file, delimiter=',')
                line_count = 0
                for row in csv_reader:
                    i = 0
                    L = len(row)
                    while (i < L-1):
                        if (line_count == 0):
                            xAxisCube.append(row[i])
                        elif (line_count == 1):
                            yAxisCube.append(row[i])
                        elif (line_count == 2):
                            zAxisCube.append(row[i])
                        i = i+1
                    line_count += 1
    except:
        print("No Cube Axes", item[0], item[1])
        err = 1 #whoops.        
    try:
        with open(cubeGeofpIRaxes) as csv_file: #remember to tab.
                csv_reader = csv.reader(csv_file, delimiter=',')
                line_count = 0
                for row in csv_reader:
                    i = 0
                    L = len(row)
                    while (i < L-1):
                        if (line_count == 0):
                            xAxisGeoIR.append(row[i])
                        elif (line_count == 1):
                            yAxisGeoIR.append(row[i])
                        elif (line_count == 2):
                            zAxisGeoIR.append(row[i])
                        i = i+1
                    line_count += 1
    except:
        print("No Geo Axes", item[0], item[1])
        err = 1 #whoops.
        
    #We now have an x, y, and z axis. x and y axes are just ordinal, but the z axis contains wavelength in microns.
    #The lengths of these arrays tell us how to extract the data.
    
    cubeData = [[[0 for x in range(len(zAxisCube))] for x in range(len(yAxisCube))] for x in range(len(xAxisCube))]
    geoIRData = [[[0 for x in range(len(zAxisGeoIR))] for x in range(len(yAxisGeoIR))] for x in range(len(xAxisGeoIR))]
    
    #The above holds the data of the cube itself. 
    try:
        with open(filepath) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            i, j, k = 0, 0, 0
            for row in csv_reader:
                while (i < len(xAxisCube)):
                    cubeData[i][j][k] = float(row[i])
                    if (math.isnan(cubeData[i][j][k])):
                        cubeData[i][j][k] = 0 #We set nans to zero to allow plotting to take place, careful!
                    elif (cubeData[i][j][k] < 0):
                        cubeData[i][j][k] = 0 #Negative values are nonsense.
                    elif (cubeData[i][j][k] > 1):
                        cubeData[i][j][k] = 1 #Make saturation obvious? Keep it from overloading. 
                    i = i + 1
                i = 0
                j = j + 1
                if (j >= len(yAxisCube)):
                    j = 0
                    k = k + 1
                line_count += 1
    except:
        print("No Cube File (how?)", item[0], item[1])
        err = 1 #whoops.
    try:
        with open(cubeGeofpIR) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            i, j, k = 0, 0, 0
            for row in csv_reader:
                while (i < len(xAxisGeoIR)):
                    geoIRData[i][j][k] = float(row[i])
                    if (math.isnan(geoIRData[i][j][k])):
                        geoIRData[i][j][k] = 0 #We set nans to zero to allow plotting to take place, careful!
                    elif (geoIRData[i][j][k] < -1000):
                        geoIRData[i][j][k] = 0 #The default value is an extremely negative number. Scrub it.
                    i = i + 1
                i = 0
                j = j + 1
                if (j >= len(yAxisGeoIR)):
                    j = 0
                    k = k + 1
                line_count += 1
    except:
        print("No Geo File", item[0], item[1])
        err = 1 #Whoops.
    #The data is now read in.
    #Now we can do stuff with it. So for the test let's just print a value from it.
    if (err == 0):
        if ((len(xAxisCube) > 4) and (len(yAxisCube) > 4)):
            newCubeList.append([item[0],item[1]])
        else:
            print("SNIPPED NOODLE!", item[0], item[1], len(xAxisCube), len(yAxisCube))
        
print(newCubeList)